import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession, DataFrameWriter
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, round
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, dense_rank
from pyspark.sql.functions import when
import pandas as pd

create local SparkSession

In [2]:
spark = SparkSession.builder.master("local[1]").appName("my_spark_app").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


read csv with inferschema

In [3]:
%%time
df = spark.read.options(header=True, inferschema=True).csv("ds_salaries.csv")

CPU times: user 2.36 ms, sys: 2.28 ms, total: 4.64 ms
Wall time: 2.81 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
%%time
df = spark.read.options(header=True, inferschema=True).csv("ds_salaries.csv")

CPU times: user 4.37 ms, sys: 401 µs, total: 4.77 ms
Wall time: 281 ms


write schema of scv on screen

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [6]:
schema = StructType([ \
StructField("id", IntegerType(), True), \
StructField("work_year", IntegerType(), True), \
StructField("experience_level", StringType(), True), \
StructField("employment_type", StringType(), True), \
StructField("job_title", StringType(), True), \
StructField("salary", IntegerType(), True), \
StructField("salary_currency", StringType(), True), \
StructField("salary_in_USD", IntegerType(), True), \
StructField("employment_residence", StringType(), True), \
StructField("remote_ratio", IntegerType(), True), \
StructField("company_location", StringType(), True), \
StructField("company_size", StringType(), True) \
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
%%time
df1 = spark.read.schema(schema).csv("ds_salaries.csv")

CPU times: user 4.35 ms, sys: 337 µs, total: 4.68 ms
Wall time: 41.4 ms


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_USD: integer (nullable = true)
 |-- employment_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df1.show()

+----+---------+----------------+---------------+--------------------+--------+---------------+-------------+--------------------+------------+----------------+------------+
|  id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_USD|employment_residence|remote_ratio|company_location|company_size|
+----+---------+----------------+---------------+--------------------+--------+---------------+-------------+--------------------+------------+----------------+------------+
|null|     null|experience_level|employment_type|           job_title|    null|salary_currency|         null|  employee_residence|        null|company_location|company_size|
|   0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                  DE|           0|              DE|           L|
|   1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                  JP|

print data in dataframe using display(df.toPandas())

In [10]:
display(df1.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_USD,employment_residence,remote_ratio,company_location,company_size
0,NaN,NaN,experience_level,employment_type,job_title,NaN,salary_currency,NaN,employee_residence,NaN,company_location,company_size
1,0.0,2020.0,MI,FT,Data Scientist,70000.0,EUR,79833.0,DE,0.0,DE,L
2,1.0,2020.0,SE,FT,Machine Learning Scientist,260000.0,USD,260000.0,JP,0.0,JP,S
3,2.0,2020.0,SE,FT,Big Data Engineer,85000.0,GBP,109024.0,GB,50.0,GB,M
4,3.0,2020.0,MI,FT,Product Data Analyst,20000.0,USD,20000.0,HN,0.0,HN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
603,602.0,2022.0,SE,FT,Data Engineer,154000.0,USD,154000.0,US,100.0,US,M
604,603.0,2022.0,SE,FT,Data Engineer,126000.0,USD,126000.0,US,100.0,US,M
605,604.0,2022.0,SE,FT,Data Analyst,129000.0,USD,129000.0,US,0.0,US,M
606,605.0,2022.0,SE,FT,Data Analyst,150000.0,USD,150000.0,US,100.0,US,M


create df_job_title that consists from all job_titles without duplicates

In [11]:
df_job_title = df1.select("job_title").dropDuplicates()

print all rows from df_job_titles without truncating jobs

In [12]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [13]:
df_analytic = df1.groupBy(["job_title"]) \
.agg( \
     avg("salary_in_usd").alias("avg_salary"), \
     min("salary_in_usd").alias("min_salary"), \
     max("salary_in_usd").alias("max_salary") \
    )

print all rows from df_analytic without trancating jobs

In [14]:
df_analytic.show(truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|Lead Data Engineer                      |139724.5          |56000     |276000    |
|Head of Machine Learning                |79039.0           |79039     |79039     |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |92203.0           |19609     |170000    |
|Data Engineering Manager                |123227.2          |59303     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
window_spec = Window.orderBy(col("avg_salary").desc())
df_analytic = df_analytic.withColumn("row_id", dense_rank().over(window_spec))

print all data from df_analytic

In [16]:
df_analytic.show(truncate=False)

23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+---------

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [17]:
df_analytic = df_analytic.select("row_id", "job_title", round("avg_salary", 2).alias("avg_salary"), "min_salary", "max_salary")

print df_analytic now

In [18]:
df_analytic.show(truncate=False)

23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/19 18:17:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+--

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [19]:
exp_window_spec = Window.partitionBy("experience_level")
df_exp_lvl = df1.withColumn("biggest_salary", max("salary_in_usd").over(exp_window_spec))

print here df_exp_lvl

In [20]:
df_exp_lvl.toPandas()

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_USD,employment_residence,remote_ratio,company_location,company_size,biggest_salary
0,5.0,2020.0,EN,FT,Data Analyst,72000.0,USD,72000.0,US,100.0,US,L,250000.0
1,10.0,2020.0,EN,FT,Data Scientist,45000.0,EUR,51321.0,FR,0.0,FR,S,250000.0
2,12.0,2020.0,EN,FT,Data Scientist,35000.0,EUR,39916.0,FR,0.0,FR,M,250000.0
3,16.0,2020.0,EN,FT,Data Engineer,4450000.0,JPY,41689.0,JP,100.0,JP,S,250000.0
4,18.0,2020.0,EN,FT,Data Science Consultant,423000.0,INR,5707.0,IN,50.0,IN,M,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,602.0,2022.0,SE,FT,Data Engineer,154000.0,USD,154000.0,US,100.0,US,M,412000.0
604,603.0,2022.0,SE,FT,Data Engineer,126000.0,USD,126000.0,US,100.0,US,M,412000.0
605,604.0,2022.0,SE,FT,Data Analyst,129000.0,USD,129000.0,US,0.0,US,M,412000.0
606,605.0,2022.0,SE,FT,Data Analyst,150000.0,USD,150000.0,US,100.0,US,M,412000.0


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [21]:
df_best = df_exp_lvl.select("id", "experience_level", "biggest_salary", "employment_residence").where(col("salary_in_usd") == col("biggest_salary"))

print df_best

In [22]:
df_best.show()

+---+----------------+--------------+--------------------+
| id|experience_level|biggest_salary|employment_residence|
+---+----------------+--------------+--------------------+
| 37|              EN|        250000|                  US|
|252|              EX|        600000|                  US|
| 33|              MI|        450000|                  US|
| 97|              MI|        450000|                  US|
| 63|              SE|        412000|                  US|
+---+----------------+--------------+--------------------+



drop duplicates if exist by experience_level

In [23]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [24]:
df_best.show()

+---+----------------+--------------+--------------------+
| id|experience_level|biggest_salary|employment_residence|
+---+----------------+--------------+--------------------+
| 37|              EN|        250000|                  US|
|252|              EX|        600000|                  US|
| 33|              MI|        450000|                  US|
| 63|              SE|        412000|                  US|
+---+----------------+--------------+--------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [25]:
df_new_best = df_best.select("experience_level", "biggest_salary", "employment_residence")
df_new_best = df_new_best.withColumn("experience_level", \
                                     when(df_new_best.experience_level == "MI", "Middle")
                                     .when(df_new_best.experience_level == "SE", "Senior")
                                     .otherwise("Null"))

print df_new_best

In [26]:
df_new_best.show()

+----------------+--------------+--------------------+
|experience_level|biggest_salary|employment_residence|
+----------------+--------------+--------------------+
|            Null|        250000|                  US|
|            Null|        600000|                  US|
|          Middle|        450000|                  US|
|          Senior|        412000|                  US|
+----------------+--------------+--------------------+



write df_new_best like 1.csv and load then it to df_final

In [27]:
import os
import shutil

def copy_csv_file(source_dir, destination_dir):
    for file in os.listdir(source_dir):
        if file.endswith(".csv"):
            csv_file = file
            break
    
    # Copy .csv file to the source directory name
    file_source_path = os.path.join(source_dir, csv_file) # Full source path of file
    new_csv_file_name = os.path.basename(source_dir.rstrip('/')) + '.csv'
    os.rename(file_source_path, new_csv_file_name)

    # Delete the source directory
    shutil.rmtree(source_dir)

df_new_best.write.format("csv").mode("overwrite").options(header="True", schema=schema, mode="overwrite").save("1")

source_directory = "1/"
destination_directory = "./1.csv"

copy_csv_file(source_directory, destination_directory)

schema = StructType() \
.add("experience_level", StringType(), True) \
.add("biggest_salary", IntegerType(), True) \
.add("employment_residence", StringType(), True) 

df_final = spark.read.options(header="True", schema=schema).csv("1.csv")

23/05/19 18:17:02 WARN ParseMode: overwrite is not a valid parse mode. Using PERMISSIVE.


print df_final

In [28]:
df_final.show()

+----------------+--------------+--------------------+
|experience_level|biggest_salary|employment_residence|
+----------------+--------------+--------------------+
|            Null|        250000|                  US|
|            Null|        600000|                  US|
|          Middle|        450000|                  US|
|          Senior|        412000|                  US|
+----------------+--------------+--------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [29]:
df_final_filtered = df_final.where(df_final.experience_level != "Null")

print df_final

In [30]:
df_final_filtered.show()

+----------------+--------------+--------------------+
|experience_level|biggest_salary|employment_residence|
+----------------+--------------+--------------------+
|          Middle|        450000|                  US|
|          Senior|        412000|                  US|
+----------------+--------------+--------------------+



last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [31]:
df_biggest_salary = df_final_filtered.select("biggest_salary").orderBy(col("biggest_salary").desc()).limit(1)
data = df_biggest_salary.collect()
biggest_salary = int(data[0][0])
print(biggest_salary)

450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system